# D3js Modal Example

Ok so I spent the past day trying to get a modal to work for a project I have at work that uses Bootstrap 4. Apparently they either depricated the external url modal support or I am too ignorant in JavaScript/HTML/CSS to get it to work (most likely... have I said how much I suck at HTML/JS/CSS!). Because I am the type of person who can't just except that something isn't going to work... I spent the past hour or so exercising my Google Foo and finally found a compatible modal example that uses D3. I don't know a lot of D3 but what I do know I really like it. It is easy to use and for the most part makes sense. So full disclosure I didn't really make this example its just adopted from [HERE](https://bl.ocks.org/lsbardel/964b454dd40bc32082a0753e9106a707). 

The following will be pretty short but I'll demonstrate the logic for integrating this into a Flask site.
### The imports

In [ ]:
<script src="https://d3js.org/d3.v4.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Faker/3.1.0/faker.min.js"></script>
<script src="https://unpkg.com/d3-require@0.4.5/build/d3-require.js"></script>
<script src="https://unpkg.com/d3-let@0.3.2/build/d3-let.js"></script>
<script src="https://unpkg.com/d3-view@0.9.3/build/d3-view.js"></script>

### The Modal Button HTML

In [ ]:
<button type="button" class="btn btn-primary" d3-on="$modal()">
    D3 Modal Button Example
</button>

# Because we are using D3, if we don't want a button and would rather have a plan text <a> tag we can. 
<a d3-on="$modal()">
    D3 Modal Plane Text Example
</a>

### The Modal JavaScript

If you want to be able to dynamically pass the JavaScript page urls, we can either include all the following JS or if we just include the declaration of the variable `modal_url` in the html file we can save the rest of the code off in a seperate file to make things easier to digest. 

In [ ]:
var modal_url = '{{ url_for('example_site.modal_example_popup') }}';

(function () {

var modal = {
    model: {
        modalTitle: "Example D3 Modal",
        modalBody: 'If you want to hard code text for the modal do it here.', 
        showModal: false,
        $showModal () {
            this.showModal = true;
        },
        $hideModal () {
            this.showModal = false;
        }
    },
    directive: {
        refresh (model, show) {
            if (!this.passes) return;
            var sel = this.sel,
                modal = sel.classed('modal');
            if (show) {
                sel.style('display', 'block').classed('show', true);
                if (modal) {
                    var height = sel.style('height');
                    sel.style('top', '-' + height);
                    this.transition(sel).ease(d3.easeExpOut).style('top', '0px');
                }
            }
            else {
                var op = sel.style('opacity'),
                    t = this.transition(sel);
                sel.classed('show', false);
                if (modal) {
                    var height = sel.style('height');
                    t.style('top', '-' + height).on('end', function () {
                        sel.style('display', 'none');
                    });
                } else
                    t.style('opacity', 0);
                t.on('end', function () {
                    sel.style('display', 'none').style('opacity', op);
                });
            }
        }
    },
    render: function () {
        return this.renderFromUrl(modal_url)
    }
};

var vm = d3.view({
    model: {
        $modal() {
            var modal = vm.select('.modal');
            if (!modal.size())
                vm.select('body').append('modal').mount(null, v => v.model.$showModal());
            else
                modal.model().$showModal();
        }
    },
    components: {
        modal: modal
    },
    directives: {
        modal: modal.directive
    }
});
vm.mount('body');
}());

### The Modal HTML Page

I had bad luck when I was playing around with it getting it to show other pages from this site. So it would seem that the html for the page to be displayed as a modal does not need to be wrapped in <html> tags, but I could be wrong as I haven't read any documentation on this yet nor have I really tried everything under the sun. This is just my brief expierence that I am trying pass along. 

In the following HTML there are a couple nobs that can be turned to adjust things like how fast the modal appears and disappears.

In [ ]:
<div>
<div class="modal" tabindex="-1" role="dialog" d3-modal="showModal" data-transition-duration="500">
  <div class="modal-dialog" role="document">
    <div class="modal-content">
      <div class="modal-header">
        <h5 class="modal-title" d3-html="modalTitle"></h5>
        <button type="button" class="close" data-dismiss="modal" aria-label="Close">
          <span aria-hidden="true" d3-on="$hideModal()">&times;</span>
        </button>
      </div>
      <div class="modal-body" d3-html="modalBody">
            <div id="construction_containter" class="text-center" >
            <img class="uc_img" src="/main/img/robots.png" alt="Matt Camp" style="width: 50%;">
            <h3>A Very Basic Example of a Modal.</h3>
            </div>
      </div>
      <div class="modal-footer">
        <button type="button" class="btn btn-secondary" d3-on="$hideModal()">Close</button>
      </div>
    </div>
  </div>
</div>
<div class="modal-backdrop fade" d3-modal="showModal" data-transition-duration="500"></div>
</div>

### Put it all together

<button type="button" class="btn btn-primary" d3-on="$modal()">
    D3 Modal Button Example
</button>

This solution is alright and will work fairly well if you only need a single modal per page. I originally thought I could pass different urls to the `modal_url` variable with D3 and it would dynamically change the page displayed on the modal. I was wrong... or more likely did something wrong again. My belief is that the function that houses the modal loads once and won't load again. I tried changing the function to be callable but there was some issue with `d3.view` I think that would throw an error on the second time the page loaded. There likely is an easy solution to this but my lack of expierence with JavaScript is precluding me at this time from using this method the way I intended to. Luckily I did finally find the prefered Bootstrap method for modals using urls. 

UPDATE: I figured out how to do this in Bootstrap 4 using iFrames. Check out the example [here] ({{ url_for('example_site.iframe_modal') }}}